In [ ]:
from cryptography.fernet import Fernet
import json
import os
import secrets
import string

# -------------------------
# KEY MANAGEMENT
# -------------------------

def load_or_create_key():
    """
    Creates a key if it doesn't exist.
    Loads it if it does.
    """
    if not os.path.exists("secret.key"):
        key = Fernet.generate_key()
        with open("secret.key", "wb") as key_file:
            key_file.write(key)
    else:
        with open("secret.key", "rb") as key_file:
            key = key_file.read()

    return Fernet(key)

cipher = load_or_create_key()


# -------------------------
# PASSWORD GENERATOR
# -------------------------

def generate_password(length=16):
    chars = string.ascii_letters + string.digits + string.punctuation
    return ''.join(secrets.choice(chars) for _ in range(length))


# -------------------------
# STORE PASSWORD
# -------------------------

def save_password(service, username, password):
    encrypted_pass = cipher.encrypt(password.encode()).decode()

    # Load existing data
    if os.path.exists("passwords.json"):
        with open("passwords.json", "r") as f:
            data = json.load(f)
    else:
        data = {}

    data[service] = {
        "username": username,
        "password": encrypted_pass
    }

    with open("passwords.json", "w") as f:
        json.dump(data, f, indent=4)

    print(f"Password saved for {service}!")


# -------------------------
# RETRIEVE PASSWORD
# -------------------------

def get_password(service):
    if not os.path.exists("passwords.json"):
        print("No saved passwords yet.")
        return

    with open("passwords.json", "r") as f:
        data = json.load(f)

    if service not in data:
        print("Service not found.")
        return

    enc_pass = data[service]["password"]
    decrypted_pass = cipher.decrypt(enc_pass.encode()).decode()

    print(f"\nService: {service}")
    print(f"Username: {data[service]['username']}")
    print(f"Password: {decrypted_pass}\n")


# -------------------------
# SIMPLE MENU
# -------------------------

def main():
    while True:
        print("\n--- PASSWORD MANAGER ---")
        print("1. Generate strong password")
        print("2. Save a password")
        print("3. Retrieve a password")
        print("4. Exit")
        choice = input("Choose an option: ")

        if choice == "1":
            length = int(input("Password length: "))
            print("Generated:", generate_password(length))

        elif choice == "2":
            service = input("Service name (e.g., Gmail): ")
            username = input("Username: ")
            use_generated = input("Generate a password? (y/n): ")

            if use_generated.lower() == "y":
                password = generate_password()
                print("Generated password:", password)
            else:
                password = input("Enter password: ")

            save_password(service, username, password)

        elif choice == "3":
            service = input("Service name: ")
            get_password(service)

        elif choice == "4":
            break

        else:
            print("Invalid choice.")


if _name_ == "_main_":
    main()